In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Objective

Build a model to automatically predict tags for a given a StackExchange question by using the text of the question.
![alt text](https://cdn.sstatic.net/Sites/stackoverflow/company/img/logos/se/se-logo.svg?v=d29f0785ebb7)

__Dataset Specs__: Over 85,000 questions

[Download Link](https://www.kaggle.com/stackoverflow/statsquestions#Questions.csv)

# Steps to Follow



1. Load Data and Import Libraries
2. Text Cleaning
3. Merge Tags with Questions
4. Dataset Prepartion
5. Text Representation
6. Model Building
    1. Define Model Architecture
    2. Train the Model
7. Model Predictions
8. Model Evaluation
9. Inference



# Load Data and Import Libraries

In [0]:
!unzip '/content/drive/My Drive/Course_Notes/Applied DL - Sequence Models/Project 1: Automatic Tagging System/statsquestions.zip'

Archive:  /content/drive/My Drive/Course_Notes/Applied DL - Sequence Models/Project 1: Automatic Tagging System/statsquestions.zip
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                
  inflating: database.sqlite         


In [0]:
#string matching
import re 

#reading files
import pandas as pd

#handling html data
from bs4 import BeautifulSoup

#visualization
import matplotlib.pyplot as plt  

pd.set_option('display.max_colwidth', 200)

In [0]:
# load the stackoverflow questions dataset
questions_df = pd.read_csv('Questions.csv',encoding='latin-1')

# load the tags dataset
tags_df = pd.read_csv('Tags.csv')

In [0]:
#print first 5 rows
questions_df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach..."
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests...."
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ..."


# Text Cleaning

Let's define a function to clean the text data.

In [0]:
def cleaner(text):

  text = BeautifulSoup(text).get_text()
  
  # fetch alphabetic characters
  text = re.sub("[^a-zA-Z]", " ", text)

  # convert text to lower case
  text = text.lower()

  # split text into tokens to remove whitespaces
  tokens = text.split()

  return " ".join(tokens)

In [0]:
# call preprocessing function
questions_df['cleaned_text'] = questions_df['Body'].apply(cleaner)

In [0]:
questions_df['Body'][1]

"<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\nareas are a lot larger than condensed\nurban areas. Is there a need to account for the area size difference?</li>\n<li>if let's say I have census data\ndating back to 4 - 5 census periods,\nhow far can i forecast it into the\nfuture?</li>\n<li>if some of the census zone change\nlightly in boundaries, how can i\naccount for that change?</li>\n<li>What are the methods to validate\ncensus forecasts? for example, if i\nhave data for existing 5 census\nperiods, should I model the first 3\nand test it on the latter two? or is\nthere another way?</li>\n<li>what's the state of practice in\nforecasting census data, and what are\nsome of the state of the art methods?</li>\n</ul>\n"

In [0]:
questions_df['cleaned_text'][1]

'what are some of the ways to forecast demographic census with some validation and calibration techniques some of the concerns census blocks vary in sizes as rural areas are a lot larger than condensed urban areas is there a need to account for the area size difference if let s say i have census data dating back to census periods how far can i forecast it into the future if some of the census zone change lightly in boundaries how can i account for that change what are the methods to validate census forecasts for example if i have data for existing census periods should i model the first and test it on the latter two or is there another way what s the state of practice in forecasting census data and what are some of the state of the art methods'

# Merge Tags with Questions

Let's now explore the tags data.

In [0]:
tags_df.head()

,Id,Tag
0,1,bayesian
1,1,prior
2,1,elicitation
3,2,distributions
4,2,normality


In [0]:
# count of unique tags
len(tags_df['Tag'].unique())

1315

In [0]:
tags_df['Tag'].value_counts()

r                            13236
regression                   10959
machine-learning              6089
time-series                   5559
probability                   4217
                             ...  
law-of-total-expectation         1
dimensions                       1
gmm                              1
auxiliary-particle-filter        1
shapley-value                    1
Name: Tag, Length: 1315, dtype: int64

In [0]:
# remove "-" from the tags
tags_df['Tag']= tags_df['Tag'].apply(lambda x:re.sub("-"," ",x))

In [0]:
# group tags Id wise
tags_df = tags_df.groupby('Id').apply(lambda x:x['Tag'].values).reset_index(name='tags')
tags_df.head()

,Id,tags
0,1,"[bayesian, prior, elicitation]"
1,2,"[distributions, normality]"
2,3,"[software, open source]"
3,4,"[distributions, statistical significance]"
4,6,[machine learning]


In [0]:
# merge tags and questions
df = pd.merge(questions_df,tags_df,how='inner',on='Id')

In [0]:
df = df[['Id','Body','cleaned_text','tags']]
df.head()

,Id,Body,cleaned_text,tags
0,6,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the two fields andrew gelman responded favorably to ...,[machine learning]
1,21,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,what are some of the ways to forecast demographic census with some validation and calibration techniques some of the concerns census blocks vary in sizes as rural areas are a lot larger than conde...,"[forecasting, population, census]"
2,22,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,how would you describe in plain english the characteristics that distinguish bayesian from frequentist reasoning,"[bayesian, frequentist]"
3,31,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....",after taking a statistics course and then trying to help fellow students i noticed one subject that inspires much head desk banging is interpreting the results of statistical hypothesis tests it s...,"[hypothesis testing, t test, p value, interpretation, intuition]"
4,36,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",there is an old saying correlation does not mean causation when i teach i tend to use the following standard examples to illustrate this point number of storks and birth rate in denmark number of ...,"[correlation, teaching]"


In [0]:
df.shape

(85085, 4)

There are over 85,000 unique questions and over 1300 tags.

# Dataset Preparation

In [0]:
# check frequency of occurence of each tag
freq= {}
for i in df['tags']:
  for j in i:
    if j in freq.keys():
      freq[j] = freq[j] + 1
    else:
      freq[j] = 1

Let's find out the most frequent tags.

In [0]:
# sort the dictionary in descending order
freq = dict(sorted(freq.items(), key=lambda x:x[1],reverse=True))

In [0]:
freq.items()

dict_items([('r', 13236), ('regression', 10959), ('machine learning', 6089), ('time series', 5559), ('probability', 4217), ('hypothesis testing', 3869), ('self study', 3732), ('distributions', 3501), ('logistic', 3316), ('classification', 2881), ('correlation', 2871), ('statistical significance', 2666), ('bayesian', 2656), ('anova', 2505), ('normal distribution', 2181), ('multiple regression', 2054), ('mixed model', 1998), ('clustering', 1952), ('neural networks', 1897), ('mathematical statistics', 1888), ('confidence interval', 1776), ('categorical data', 1703), ('generalized linear model', 1614), ('variance', 1576), ('data visualization', 1549), ('estimation', 1533), ('forecasting', 1422), ('t test', 1418), ('pca', 1395), ('sampling', 1363), ('cross validation', 1344), ('repeated measures', 1335), ('spss', 1296), ('svm', 1283), ('chi squared', 1261), ('maximum likelihood', 1209), ('predictive models', 1189), ('multivariate analysis', 1116), ('survival', 1081), ('references', 1076), (

In [0]:
# Top 10 most frequent tags
common_tags = list(freq.keys())[:10]
common_tags

['r',
 'regression',
 'machine learning',
 'time series',
 'probability',
 'hypothesis testing',
 'self study',
 'distributions',
 'logistic',
 'classification']

We will use only those questions/queries that have the above 10 tags associated with it.

In [0]:
x=[]
y=[]

for i in range(len(df['tags'])):
  
  temp=[]
  for j in df['tags'][i]:
    if j in common_tags:
      temp.append(j)

  if(len(temp)>1):
    x.append(df['cleaned_text'][i])
    y.append(temp)

In [0]:
# number of questions left
len(x)

11106

In [0]:
y[:10]

[['r', 'time series'],
 ['regression', 'distributions'],
 ['distributions', 'probability', 'hypothesis testing'],
 ['hypothesis testing', 'self study'],
 ['r', 'regression', 'time series'],
 ['r', 'time series', 'self study'],
 ['probability', 'hypothesis testing'],
 ['r', 'regression'],
 ['r', 'regression'],
 ['regression', 'logistic']]

Now we will find a suitable sequence length.

We will the input sequences to our model to the length of 100

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
 
y = mlb.fit_transform(y)
y.shape

(11106, 10)

In [0]:
y[0,:]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1])

In [0]:
mlb.classes_

array(['classification', 'distributions', 'hypothesis testing',
       'logistic', 'machine learning', 'probability', 'r', 'regression',
       'self study', 'time series'], dtype=object)

We can now split the dataset into training set and validation set. 

In [0]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(x, y, test_size=0.2, random_state=0,shuffle=True)

# Text Representation

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 

#prepare a tokenizer
x_tokenizer = Tokenizer() 

#prepare vocabulary
x_tokenizer.fit_on_texts(x_tr)

In [0]:
x_tokenizer.word_index

{'the': 1,
 'i': 2,
 'to': 3,
 'a': 4,
 'of': 5,
 'is': 6,
 'and': 7,
 'in': 8,
 'l': 9,
 'x': 10,
 'for': 11,
 'that': 12,
 'data': 13,
 'this': 14,
 't': 15,
 'have': 16,
 'y': 17,
 'with': 18,
 'model': 19,
 'it': 20,
 'are': 21,
 'be': 22,
 'my': 23,
 'as': 24,
 'on': 25,
 'e': 26,
 'p': 27,
 'if': 28,
 'can': 29,
 'n': 30,
 'but': 31,
 'not': 32,
 'm': 33,
 'or': 34,
 'r': 35,
 'how': 36,
 'regression': 37,
 'c': 38,
 'am': 39,
 's': 40,
 'from': 41,
 'test': 42,
 'what': 43,
 'would': 44,
 'b': 45,
 'so': 46,
 'time': 47,
 'there': 48,
 'using': 49,
 'which': 50,
 'an': 51,
 'do': 52,
 'one': 53,
 'each': 54,
 'value': 55,
 'use': 56,
 'by': 57,
 'some': 58,
 'variables': 59,
 'like': 60,
 'variable': 61,
 'we': 62,
 'at': 63,
 'na': 64,
 'any': 65,
 'f': 66,
 'distribution': 67,
 'two': 68,
 'values': 69,
 'set': 70,
 'you': 71,
 'all': 72,
 'function': 73,
 'fit': 74,
 'd': 75,
 'beta': 76,
 'question': 77,
 'then': 78,
 'mean': 79,
 'me': 80,
 'know': 81,
 'where': 82,
 'when'

In [0]:
len(x_tokenizer.word_index)

25312

There are around 25,000 tokens in the training dataset. Let's see how many tokens appear at least 5 times in the dataset.

In [0]:
thresh = 3

cnt=0
for key,value in x_tokenizer.word_counts.items():
  if value>=thresh:
    cnt=cnt+1

print(cnt)

12574


Over 12,000 tokens have appeared three times or more in the training set.

In [0]:
# prepare the tokenizer again
x_tokenizer = Tokenizer(num_words=cnt,oov_token='unk')

#prepare vocabulary
x_tokenizer.fit_on_texts(x_tr)

Now that we have encoded every token to an integer, let's convert the text sequences to integer sequences. After that we will pad the integer sequences to the maximum sequence length, i.e., 100.

In [0]:
#define threshold for maximum length of a setence
max_len=100

#convert text sequences into integer sequences
x_tr_seq = x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq = x_tokenizer.texts_to_sequences(x_val)

#padding up with zero 
x_tr_seq = pad_sequences(x_tr_seq,  padding='post', maxlen=max_len)
x_val_seq = pad_sequences(x_val_seq, padding='post', maxlen=max_len)

Since we are padding the sequences with zeros, we must increment the vocabulary size by one.

In [0]:
#no. of unique words
x_voc_size = x_tokenizer.num_words + 1
x_voc_size

12575

In [0]:
x_tr_seq[0]

array([1953, 5711,  416, 2023,    1,  226, 1747, 3740,  609,   43,  181,
       1953,  372,   19,  100,  416,    9, 1747, 3839,  238,   27,   27,
         27,   27,   27,   70,    6, 6919,    8, 1163,   70,    6,   43,
         43, 1802, 1802, 1802,   36,   36,   36,   36, 4308, 5410,    4,
        124,  592,  107,   22,    2, 1747, 4065,   27,   10, 1309,   10,
       6414,   10,  190,   10,  416,   10,   27,   10, 1309,   10, 6414,
         10,  190,   10,  416,   10,  456,  139,   15,    7,    2, 4610,
        164,   27,   10, 1309,   10, 6414,   10,  190,   10,  416,   10,
         27,   76,   27, 1309,   76,   27, 6414,   76,   27,  190,   76,
         27], dtype=int32)

# Model Building

In [0]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *

### Define Model Architecture

In [0]:
#sequential model
model = Sequential()

#embedding layer
model.add(Embedding(x_voc_size, 50, trainable = True, input_shape=(max_len,),mask_zero=True))

#lstm 
model.add(LSTM(128))

#dense layer
model.add(Dense(128,activation='relu')) 

#output layer
model.add(Dense(10,activation='sigmoid'))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           628750    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 738,200
Trainable params: 738,200
Non-trainable params: 0
_________________________________________________________________


In [0]:
#define optimizer and loss
model.compile(optimizer='adam',loss='binary_crossentropy')

#checkpoint to save best model during training
mc = ModelCheckpoint("weights.best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

### Train the Model

In [0]:
#train the model 
model.fit(x_tr_seq, y_tr, batch_size=128, epochs=10, verbose=1, validation_data=(x_val_seq, y_val), callbacks=[mc])

Train on 8884 samples, validate on 2222 samples
Epoch 1/10
8884/8884 [==============================] - 17s 2ms/step - loss: 0.5194 - val_loss: 0.4835

Epoch 00001: val_loss improved from inf to 0.48352, saving model to weights.best.hdf5
Epoch 2/10
8884/8884 [==============================] - 17s 2ms/step - loss: 0.4817 - val_loss: 0.4795

Epoch 00002: val_loss improved from 0.48352 to 0.47953, saving model to weights.best.hdf5
Epoch 3/10
8884/8884 [==============================] - 17s 2ms/step - loss: 0.4590 - val_loss: 0.4330

Epoch 00003: val_loss improved from 0.47953 to 0.43297, saving model to weights.best.hdf5
Epoch 4/10
8884/8884 [==============================] - 16s 2ms/step - loss: 0.4146 - val_loss: 0.4163

Epoch 00004: val_loss improved from 0.43297 to 0.41634, saving model to weights.best.hdf5
Epoch 5/10
8884/8884 [==============================] - 16s 2ms/step - loss: 0.3811 - val_loss: 0.3834

Epoch 00005: val_loss improved from 0.41634 to 0.38345, saving model to weig

# Model Predictions 

In [0]:
# load weights into new model
model.load_weights("weights.best.hdf5")

#predict probabilities
pred_prob = model.predict(x_val_seq)

In [0]:
pred_prob[0]

array([0.00962889, 0.00125104, 0.00590172, 0.58005685, 0.03403521,
       0.00300524, 0.71871006, 0.73088944, 0.01342303, 0.06519052],
      dtype=float32)

The predictions are in terms of probabilities for each of the 10 tags. Hence we need to have a threshold value to convert these probabilities to 0 or 1.

Let's specify a set of candidate threshold values. We will select the threshold value that performs the best for the validation set.

In [0]:
#define candidate threshold values
threshold  = np.arange(0,0.5,0.01)
threshold

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49])

Let's define a function that takes a threshold value and uses it to convert probabilities into 1 or 0.

In [0]:
# convert probabilities into classes or tags based on a threshold value
def classify(pred_prob,thresh):
  y_pred_seq = []

  for i in pred_prob:
    temp=[]
    for j in i:
      if j>=thresh:
        temp.append(1)
      else:
        temp.append(0)
    y_pred_seq.append(temp)

  return y_pred_seq

In [0]:
from sklearn import metrics
score=[]

#convert to 1 array
y_true = np.array(y_val).ravel() 

for thresh in threshold:
    
    #classes for each threshold
    y_pred_seq = classify(pred_prob,thresh) 

    #convert to 1d array
    y_pred = np.array(y_pred_seq).ravel()

    score.append(metrics.f1_score(y_true,y_pred))

In [0]:
# find the optimal threshold
opt = threshold[score.index(max(score))]
opt

0.33

# Model Evaluation

In [0]:
#predictions for optimal threshold
y_pred_seq = classify(pred_prob,opt)
y_pred = np.array(y_pred_seq).ravel()

In [0]:
print(metrics.classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.86      0.88     17520
           1       0.56      0.67      0.61      4700

    accuracy                           0.82     22220
   macro avg       0.73      0.77      0.75     22220
weighted avg       0.83      0.82      0.82     22220



## How to improve Model's Performance?

1. You can add more LSTM layers
2. You can pass pre-trained word embeddings
3. You can play with different optimizers
4. Try different input sequence length

In [0]:
y_pred = mlb.inverse_transform(np.array(y_pred_seq))
y_true = mlb.inverse_transform(np.array(y_val))

df = pd.DataFrame({'comment':x_val,'actual':y_true,'predictions':y_pred})

In [0]:
df.sample(10)

,comment,actual,predictions
909,i currently have hourly electricity demand data last for years where i used demand msts mydata demand seasonal period c start the plot of stl shows the data have a clear decreasing trend and seaso...,"(r, time series)","(r, time series)"
608,i know this might be a very basic question for anyone but i m not sure how to answer it correctly it was recently asked at an interview it would be great if someone could help me with answering th...,"(distributions, probability)","(probability, self study)"
874,i am trying to do a simple regression either polynomial or support vector regression for solar power prediction as a project to learn machine learning however the data i have is time series data o...,"(machine learning, regression, time series)","(logistic, machine learning, regression)"
560,i am working on a model based on logistic regression with a binary response variable and my data consists of ratios of integers number of positive observations out of the total number of observati...,"(logistic, regression)","(regression, self study)"
1225,i realize that similar questions have already been asked and answered but i am in need of a bit more detail and specific advice as i am new to pca and statistical methods in general my question is...,"(r, regression)","(logistic, regression)"
1989,i am setting up a linear model in r and need help understanding the significance codes when one of my independent variables is a factor i e dummy variable for each possible value for a scalar inde...,"(r, regression)","(logistic, r, regression)"
938,how do i test if two time series coefficients differ significantly from one another i feel like this should be pretty simple should i just use the estimates standard errors and calculate a z score...,"(hypothesis testing, time series)","(r, regression, time series)"
1259,i stumbled upon the following problem given n dice with m faces with values to m and a number x what is the probability that the sum of the numbers on the m dice is greater than or equal to x that...,"(distributions, probability)","(distributions, probability, self study)"
730,much is being made today about the effect of exercise on bmi in young people today a study was conducted to assess the factors affecting exercise study participants were asked how many minutes per...,"(distributions, probability)","(logistic, regression)"
254,i m trying to understand the bayes classifier i don t really understand its purpose or how to apply it but i think i understand the parts of the formula p y j mid x x if i m correct it s asking fo...,"(probability, r)","(classification, machine learning, r)"


# Inference

In [0]:
def predict_tag(comment):  
  text=[]

  #preprocess  
  text = [cleaner(comment)]

  #convert to integer sequences
  seq = x_tokenizer.texts_to_sequences(text)

  #pad the sequence
  pad_seq = pad_sequences(seq,  padding='post', maxlen=max_len)

  #make predictions
  pred_prob = model.predict(pad_seq)
  classes = classify(pred_prob,opt)[0]
  
  classes = np.array([classes])
  classes = mlb.inverse_transform(classes)  
  return classes

In [0]:
comment = "For example, in the case of logistic regression, the learning function is a Sigmoid function that tries to separate the 2 classes"

print("Comment:",comment)
print("Predicted Tags:",predict_tag(comment))

Comment: For example, in the case of logistic regression, the learning function is a Sigmoid function that tries to separate the 2 classes
Predicted Tags: [('classification', 'logistic', 'regression')]
